In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')
%matplotlib inline

In [2]:
with open('Manual WorkItems alltypes Security.csv', encoding='ascii', errors='ignore') as infile:
    dataset1 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])
with open('Manual WorkItems alltypes nonSecurity.csv', encoding='ascii', errors='ignore') as infile:
    dataset2 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])
    
dataset1 = dataset1.sample(frac = 0.45, random_state= 123).reset_index(drop = True)
dataset1.info()

dataset1['IsSecured'] = 1
dataset2['IsSecured'] = 0

dataset = pd.concat([dataset1, dataset2]).reset_index(drop = True)
dataset = dataset.sample(frac = 1).reset_index(drop = True)

dataset.dropna(axis=0, how='any', inplace = True)
dataset=dataset.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63262 entries, 0 to 63261
Data columns (total 3 columns):
Title         63261 non-null object
Security      63262 non-null object
SourceLink    63262 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


In [3]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
#type(my_stopwords),my_stopwords

In [11]:
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
import os, sys
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return " ".join(allparts)

In [6]:
#I will now apply the splitall function on the dataset.
dataset['Parsed Title'] = dataset['Title'].apply(splitall)
#dataset.head(100)

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def clean_text(text):
    text = "".join([char if char.isalpha() else " " for char in text.lower()]).split()
    #text = [ps.stem(word) for word in text]
    #text = " ".join([word for word in text if (word not in my_stopwords and len(word) > 2)])
    text = " ".join([word for word in text if (len(word) > 2)])
    return text.lower()

In [8]:
%time dataset['Clean Title'] = dataset['Parsed Title'].apply(clean_text)

Wall time: 2.1 s


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer,CountVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
X, y = dataset['Clean Title'], dataset['IsSecured']
cvectorizer = CountVectorizer(min_df=20, stop_words='english')
#tv = TfidfVectorizer(min_df=0.001, max_df= 1.0, use_idf=True,ngram_range=(1,3))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
cvz = cvectorizer.fit_transform(X_train)#.toarray()
X_val = cvectorizer.transform(X_val)#.toarray()
vocab = cvectorizer.get_feature_names()
print(len(vocab))


3904


In [21]:
cvz

<97646x3904 sparse matrix of type '<class 'numpy.int64'>'
	with 716343 stored elements in Compressed Sparse Row format>

In [22]:
X_val.shape,X_train.shape

((24412, 3904), (97646,))

In [31]:
import lda

# train an LDA model
lda_model = lda.LDA(n_topics=33, n_iter=500)
X_topics = lda_model.fit_transform(cvz)

INFO:lda:n_documents: 97646
INFO:lda:vocab_size: 3904
INFO:lda:n_words: 763196
INFO:lda:n_topics: 33
INFO:lda:n_iter: 500
C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -9084161
INFO:lda:<10> log likelihood: -6398433
INFO:lda:<20> log likelihood: -5857462
INFO:lda:<30> log likelihood: -5710124
INFO:lda:<40> log likelihood: -5641363
INFO:lda:<50> log likelihood: -5604766
INFO:lda:<60> log likelihood: -5580474
INFO:lda:<70> log likelihood: -5562223
INFO:lda:<80> log likelihood: -5549584
INFO:lda:<90> log likelihood: -5540420
INFO:lda:<100> log likelihood: -5532571
INFO:lda:<110> log likelihood: -5525362
INFO:lda:<120> log likelihood: -5517430
INFO:lda:<130> log likelihood: -5511407
INFO:l

In [32]:
import numpy

threshold = 0.5
_idx = np.amax(X_topics, axis=1) > threshold  # idx of doc that above the threshold
X_topics = X_topics[_idx]

In [33]:
from sklearn.manifold import TSNE

# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results 
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')

# 20-D -> 2-D
tsne_lda = tsne_model.fit_transform(X_topics)


import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool

n_top_words = 5 # number of keywords we show


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 28125 samples in 0.237s...
[t-SNE] Computed neighbors for 28125 samples in 55.892s...
[t-SNE] Computed conditional probabilities for sample 1000 / 28125
[t-SNE] Computed conditional probabilities for sample 2000 / 28125
[t-SNE] Computed conditional probabilities for sample 3000 / 28125
[t-SNE] Computed conditional probabilities for sample 4000 / 28125
[t-SNE] Computed conditional probabilities for sample 5000 / 28125
[t-SNE] Computed conditional probabilities for sample 6000 / 28125
[t-SNE] Computed conditional probabilities for sample 7000 / 28125
[t-SNE] Computed conditional probabilities for sample 8000 / 28125
[t-SNE] Computed conditional probabilities for sample 9000 / 28125
[t-SNE] Computed conditional probabilities for sample 10000 / 28125
[t-SNE] Computed conditional probabilities for sample 11000 / 28125
[t-SNE] Computed conditional probabilities for sample 12000 / 28125
[t-SNE] Computed conditional probabilities for sa

In [34]:
_lda_keys = []
for i in range(X_topics.shape[0]):
    _lda_keys +=  X_topics[i].argmax(),

topic_summaries = []
topic_word = lda_model.topic_word_  # all topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1] # get!
    topic_summaries.append(' '.join(topic_words)) # append!

In [35]:
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5",
    "#1f7874", "#aec777", "#fffeee", "#ffbbbb", "#2caaac",
    "#98dfaa", "#d66666", "#ff9996", "#9499bd", "#c55445",
    "#8c5467", "#c49ccc", "#e33332", "#f77dd2", "#7ffeef",
    "#c7c9d9", "#bc4321", "#dbbbed", "#17bbef", "#9eeea5",
    "#1fc7bb", "#ae8888", "#ffebbe", "#aebf18", "#2caaac",
    "#981f2a", "#d66ee7", "#ff9996", "#91eebd", "#c555d5",
    "#8c321b", "#c4444a", "#eeefff", "#fbbdd2", "#77777f",
    "#c7b7b7", "#bc1222", "#dbae8d", "#17cccf", "#9eddd5",
    "#c7f7f7", "#bace22", "#db818d", "#17bfff", "#9eaaa5"
])

In [36]:
title = '33 groups LDA viz'
num_example = len(X_topics)

plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.circle(x=tsne_lda[:,0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example]
                 )

#source = ColumnDataSource(data=dict(x= tsne_lda[:, 0], y=tsne_lda[:, 1]))
#plot_lda.circe(x='x', y='x',source=source)
#plot_lda

GlyphRenderer(id='1f54c4c2-5690-4e9b-bf47-1f43c0b3d896', ...)

In [37]:
# randomly choose a news (within a topic) coordinate as the crucial words coordinate
topic_coord = np.empty((X_topics.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
    if not np.isnan(topic_coord).any():
        break
    topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

# plot crucial words
for i in range(X_topics.shape[1]):
    plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key"}

# save the plot
save(plot_lda, '{}.html'.format(title))

C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\io\saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\io\saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\inpganavar\\Desktop\\MS\\63 groups LDA viz.html'